In [7]:
%%html
<style>
    .main-title{
        text-align:center;
        padding: 30px;
        border-top: 12px double darkblue;
        border-bottom: 4px solid darkblue;
        font-size: 2.7em;
        font-family: georgia,garamond,serif;
        text-transform: uppercase;
        background-color: #e6faff;
    }
    .notebook-description{
        text-align:justify;
        margin-left: 100px;
        margin-right: 100px;
        line-height: 1.25;
        
    }
    .project-scope{
        text-align: justify;
        line-height: 1.5;
    }
    .features-used-description > dt::before {
        content: "•";
        font-size: 17px;
        color: darkblue;
        display: inline-block; 
        width: 0.7em;
    }
    .features-used-description > dt{
        font-size: 1.2em;
    }
    .features-used-description > dd{
        margin-left: 30px;
    }
    .project-structure{
        text-align: justify;
        line-height: 1.5;
    }
    .section-heading{
#         background-color: #e6faff;
        font-size: 2em;
        padding-left: 10px;
        text-align: left;
        border-left: 3px solid darkblue;
        height: 50px;
        font-family: cursive;
    }
    .section-content{
        margin-top: 10px;
        text-align: justify;
        line-height: 1.5;
    }
    .output_subarea iframe{
        width: 950px !important;
    }
</style>

<div class="main-title"> B<sub>1</sub> interactive filtering notebook </div>

<div class="notebook-description"><i><b>Project Jupyter together with Jupyter Notebooks exist to develop and enable open-source software with open standards for interactive computing across a wide range of programming lanugages. The purpose of this interactive notebook is to demonstrate some of the features that Jupyter Notebook provides for its users.</b></i></div>

<div class="project-scope">
Jupyter Notebooks enable its users the possibility to integrate code written in more than 40 programming languages, the output of the code and a narrative that gives it the feel of a real notebook. This makes it possible for scientific calculations of multiple scientific areas and experiments to be easily shared, reproduced and verified. <br/>
    To fulfill this purpose there are many features that Jupyter supports to provide data visualization, displaying equations, tables, figures and also, live cells. <br/>
    This paper uses a reaserch topic from **Magnetic Resonance Imaging (MRI)** to demonstrate the following features of Jupyter Notebooks:
    <br/><br/>
    <dl class="features-used-description">
        <dt>Mixing code and narrative, displaying equations</dt>
        <dd>These are some of the basic features of Jupyter Notebooks<dd>
        <dt>Script of Scripts(SoS) polyglot notebooks</dt>
        <dd>SoS Polyglot Notebook is a Jupyter Notebook with a SoS kernel.<br/>
            SoS Notebook serves as a super kernel to all other Jupyter kernels and allows the use of multiple kernels in one Jupyter notebook.<br/>
            This project uses kernels for MATLAB/Octave and Python.</dd>
        <dt>Plotly Dash</dt>
        <dd>Dash is a productive Python framework for building web applications. It's written on top of Flask, Plotly.js, and React.js. Dash is ideal for building data visualization apps with highly custom user interfaces in pure Python. It's particularly suited for anyone who works with data in Python.</dd>
        <dt>Plotly express</dt>
        <dd>Plotly Express is a terse, consistent, high-level API for rapid data exploration and figure generation.</dd>
    </dl>

Apart from this, this Notebook will use code provided by the project <a href="https://qmrlab.org/">qMRLab</a>. qMRLab is an open-source software for quantitative analysis of MRI records. <br/> 
The main goal of the qMRLab project is to provide the community with software that makes data fitting, simulation and protocol optimization as easy as possible for a myriad of different quantitative models.
</div>



<div class="project-structure">The following sections will contain brief content about the B<sub>1</sub> double angle mapping method and the filtering of the resulting images. 
First there will be an introduction to B<sub>1</sub> field, then we'll explain the double angle mapping method. Afterwards, an explanation of the filtering module by qMRLab will be provided, followed by MATLAB code implemented to calculate the B<sub>1</sub> DAM maps and their filtered versions. After the code, a new section will give an introduction to the structure of a Plotly Dash application and right after that the dashboard implementation code will be displayed. After the code, the visual output of the whole project will be visible to the user and will enable the interactivity to display different filter maps. At the end, a brief conclusion will be presented, followed by acknowledgement and references sections.
</div>

<div class="B1-field">
    <span class="section-heading"> <i> What is B<sub>1</sub> field? </i></span>
    <div class="section-content">
        The transmit radio-frequency (RF) field amplitude (“B<sub>1</sub><sup>+</sup>”, but more frequently written simply as “B<sub>1</sub>” in the context of quantitative MRI imaging) is a quantity that directly impacts the actual flip angle that magnetization in a voxel rotates due to on-resonance RF pulses. Spatial inhomogeneity of B<sub>1</sub> leads to spins across the sample experiencing different flip angles, which can lead to differences in image signal intensity throughout a homogeneous sample. Although <sub>B1</sub> can refer to the actual RF magnetic field amplitude (on the order of microTeslas), in the context of quantitative MRI it’s more frequently represented as a normalized correction factor of the nominal flip angle set by the user at the scanner (&alpha;<sub>actual</sub> = B<sub>1</sub>·&alpha;<sub>nominal</sub>). 
    </div>
</div>

<div class="B1-field">
    <span class="section-heading"> <i> What is B<sub>1</sub> mapping? </i></span>
    <div class="section-content">
        
    </div>
</div>

In [8]:
%% MATLAB/OCTAVE CODE

% Adds qMRLab to the path of the environment
cd qMRLab
startup

error: qMRLab: No such file or directory
loading struct
loading io
loading statistics
loading optim
loading image


In [9]:
%% MATLAB/OCTAVE CODE

% Download brain MRI data
cmd = ['curl -L -o b1_map.zip https://osf.io/mw3sq/download/'];
[STATUS,MESSAGE] = unix(cmd);
unzip('b1_map.zip');

In [10]:
%% MATLAB/OCTAVE CODE

% Create repository for filtered images if such repository doesn't previously exist
fn = fullfile('ResultsBank');
if ~exist(fn, 'dir')
    mkdir(fn);
end

In [11]:
%% MATLAB/OCTAVE CODE

clear all

% Create a Model variable that represents an object from the b1_dam class.
Model = b1_dam;

% Format data structure so that they may be fit by the model
data = struct();
data.SFalpha = double(load_nii_data('SFalpha.nii.gz')); % Load data for image with excitation angle value of alpha
data.SF2alpha  = double(load_nii_data('SF2alpha.nii.gz')); % Load data for image with excitation angle value of 2*alpha
data.Mask=double(load_nii_data(['..' filesep 'SF60-MASK.nii.gz'])); % Set Mask for the model

% Setting types of possible mapping methods, filtering_types, dimensions and order/size of filters.
% The only implemented method for now is the DAM (Double Angle Mapping) method for B1 mapping.
method_types = {'DAM', 'Method2', 'Method3'};
filtering_types = {'spline', 'polynomial', 'gaussian', 'median'};
filtering_dimensions = {'2D', '3D'};
filtering_orders = {2, 4, 6};
size_methods = {'gaussian', 'median'}; % List of types that use size paramether, and not order

% Create separate directory inside ResultsBank folder for each mapping method
for mm= 1:length(method_types)
    method_type= method_types{mm}
    fn = fullfile('ResultsBank', method_type);
    if ~exist(fn, 'dir')
        mkdir(fn);
    end
end

% Using model to generate map and filtered map for different combinations of filtering parameters
% Saving the result files in separate folders of destination_folder_path
destination_folder_path = ['ResultsBank' filesep 'DAM'];
for i= 1:length(filtering_types)
    filter_type= filtering_types{i};
    for j = 1: length(filtering_dimensions)
        dimension = filtering_dimensions{j};
        for k = 1: length(filtering_orders)
            order = filtering_orders{k};
            
            % Adjust model for filtering
            Model.options.Smoothingfilter_Type = filter_type;
            Model.options.Smoothingfilter_Dimension = dimension;
            
            % If filter type is gaussian or median we use the filtering size, otherwise we use filtering order
            if sum(strcmp(size_methods, filter_type))>0
                Model.options.Smoothingfilter_sizex = order;
                Model.options.Smoothingfilter_sizey = order;
                Model.options.Smoothingfilter_sizez = order;
            else 
                Model.options.Smoothingfilter_order = order;
            end
            
            % Try to fit data to the model and save output in FitResults variable
            try 
                FitResults = FitData(data,Model);
            catch
                continue;
            end
            
            # Save FitResults in correct location on disk
            B1_raw = FitResults.B1map_raw;
            B1_filtered = FitResults.B1map_filtered;
            typepath = fullfile(destination_folder_path, filter_type);
            mkdir(typepath, [dimension filesep num2str(order)]);
            try
                fulldestination = fullfile(typepath, [dimension filesep num2str(order) filesep 'B1_raw.mat']);
                save(fulldestination, 'B1_raw', '-v7');
                fulldestination = fullfile(typepath, [dimension filesep num2str(order) filesep 'B1_filtered.mat']);
                save(fulldestination, 'B1_filtered', '-v7');
            catch Message
                Message
                continue;
            end
            
        end
    end
end

ans = 1
    load_nii_hdr>read_header at line 114 column 5
    load_nii_hdr at line 52 column 14
    load_nii at line 184 column 53
    load_nii_data at line 3 column 5
ans = 1
    load_nii_hdr>read_header at line 114 column 5
    load_nii_hdr at line 52 column 14
    load_nii at line 184 column 53
    load_nii_data at line 3 column 5
ans = 1
    load_nii_hdr>read_header at line 114 column 5
    load_nii_hdr at line 52 column 14
    load_nii at line 184 column 53
    load_nii_data at line 3 column 5
method_type = DAM
method_type = Method2
method_type = Method3
=============== qMRLab::Fit ======================
Operation has been started: b1_dam
Elapsed time is 0.120755 seconds.
Operation has been completed: b1_dam
=============== qMRLab::Fit ======================
Operation has been started: b1_dam
Elapsed time is 0.0862482 seconds.
Operation has been completed: b1_dam
=============== qMRLab::Fit ======================
Operation has been started: b1_dam
Elapsed time is 0.0897961 seconds

In [12]:
# PYTHON CODE
# Module imports

import nibabel as nib

import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
from jupyter_plotly_dash import JupyterDash

import plotly_express as px
import plotly.graph_objs as go

import matplotlib.pyplot as mpl
from matplotlib import cm
import numpy as np

from scipy.io import loadmat
import os


filesep = os.path.sep;

#  Load image with excitation angle value of alpha for display on the final figure 
SFalpha = nib.load('qMRLab' + filesep + 'SFalpha.nii.gz');
data_SFalpha = SFalpha.get_fdata();
fig_SFalpha = px.imshow(data_SFalpha, color_continuous_scale='jet', zmin=0.9, zmax=1.05);
fig_SFalpha.data[0].showscale=False;
fig_SFalpha.data[0].coloraxis=None;
fig_SFalpha.update_layout(width=200, height=250, margin=dict(l=0, r=0, b=10, t=50), title_text= "SFalpha");
fig_SFalpha.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False);

#  Load image with excitation angle value of 2*alpha for display on the final figure 
SF2alpha = nib.load('qMRLab' + filesep + 'SF2alpha.nii.gz');
data_SF2alpha = SF2alpha.get_fdata();
fig_SF2alpha = px.imshow(data_SF2alpha, color_continuous_scale='jet', zmin=0.9, zmax=1.05);
fig_SF2alpha.data[0].showscale=False;
fig_SF2alpha.data[0].coloraxis=None;
fig_SF2alpha.update_layout(width=200, height=250, margin=dict(l=0, r=0, b=10, t=50), title_text="SF2alpha");
fig_SF2alpha.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False);

# Defining the data for all the dropdown lists in a dictionary
dropdown_data = {'B1 mapping method' : ['DAM', 'Method2', 'Method3'],
        'Filter type':['Spline', 'Polynomial', 'Gaussian', 'Median'],
        'Dimension':['2D', '3D'], 
        'Filtering order': [2,4,6]}

# Create a JupyterDash application
app = JupyterDash("b1_dash_app")

# Defining the Dash application layout by adding html and dcc components
# The first component is a Div tag that serves as a wrapper.
app.layout = html.Div(
    [      
        # Defining the dashboard header
        html.H1([html.I("B"),
                 html.I(html.Sub("1")), 
                 html.I(" mapping - interactive dashboard")],
                 style={"text-align":"center","font-family":"georgia,garamond,serif", "font-weight" : "700"}),
        
        # Describing the purpose of the dashboard in another header tag.
        html.H3("This is an interactive dashboard that allows the user to choose what output he would like displayed " +
                "based on different filtering parameters. ", 
                style={"text-align":"center","font-weight":"200","font-family":"georgia,garamond,serif", 
                       "color":"darkblue", "padding-left":"15%",  "padding-right":"15%"}),
        
        # Creating a Div tag that will contain all the dropdown lists for user input
        html.Div(
            [
                # Iterating through dictionary variable: dropdown_data and creating dropdowns with corresponding labels and values
                html.P([d + ":", dcc.Dropdown(id=d, options=[dict(label=m, value=m) for m in dropdown_data[d]], value=dropdown_data[d][0])], 
                      style={"width":"23.5%", "display":"inline-block", "margin-right":"5px"})
                for d in dropdown_data.keys()
            ],
            style={"width": "100%"},
        ),
        
        # Listing the graphs that should be displayed.
        # The graphs SFalpha and SF2alpha will be loaded statically and will display the input images for the method.
        # The graphs map_raw and map_filtered will be loaded depanding on the user input.
        # The graph map_raw will display the raw map after applying the mapping method on the input dataset.
        # The graph map_filtered will display the filtered map for the mapping method,
        # based on the filtering parameters chosen in the available dropdown lists.
        dcc.Graph(id="SFalpha", figure = fig_SFalpha, style={"width": "25%", "display": "inline-block"}),
        dcc.Graph(id="SF2alpha", figure = fig_SF2alpha, style={"width": "25%", "display": "inline-block"}),
        dcc.Graph(id="map_raw", style={"width": "24%", "display": "inline-block"}),
        dcc.Graph(id="map_filtered", style={"width": "25%", "display": "inline-block"}),
    ]
)

# Adding interactivity to the Dash application by defining a callback function.
# The function takes as input the values from all the dropdown lists and returns two figures: the raw map and the filtered map.
@app.callback([Output("map_raw", "figure"), Output("map_filtered", "figure")], [Input(d, "value") for d in dropdown_data.keys()])
def make_figure(method_type, filtering_type, dimension, order):

    # Defining path to raw and filtered files
    path = '.' + filesep + 'qMRLab' + filesep + 'ResultsBank' + filesep + method_type + filesep + filtering_type.lower() + filesep + dimension + filesep + str(order) + filesep;
    file_path_raw = path  + 'B1_raw.mat';
    file_path_filtered = path + 'B1_filtered.mat';
    
    # Creating a graph figure for the raw map and setting its options
#     title_for_figure = "Type: %s, dimension: %s and order %s" % (filtering_type, dimension, order)
    title_for_figure = "B<sub>1</sub> " + method_type + " raw map";
    B1_raw = loadmat(file_path_raw);
    b1_raw_data = B1_raw['B1_raw'];
    fig_raw = px.imshow(b1_raw_data, color_continuous_scale='jet', zmin=0.9, zmax=1.05);
    fig_raw.update_layout(width=200, height=250, margin=dict(l=0, r=0, b=10, t=50), title_text= title_for_figure, coloraxis_showscale=False)
    fig_raw.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)

    # Creating a graph figure for the filtered map and setting its options
#     title_for_figure = "Type: %s, dimension: %s and order %s" % (filtering_type, dimension, order)
    title_for_figure = "B<sub>1</sub> " + method_type + " filtered map"
    B1_filtered = loadmat(file_path_filtered);
    b1_filtered_data = B1_filtered['B1_filtered'];
    fig_filtered = px.imshow(b1_filtered_data, color_continuous_scale='jet', zmin=0.9, zmax=1.05);
    fig_filtered.update_layout(width=200, height=250, margin=dict(l=0, r=0, b=10, t=50), title_text= title_for_figure, coloraxis_showscale=False)
    fig_filtered.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)
    
    return fig_raw, fig_filtered 

app